In [1]:
import pandas as pd
import numpy as np
from HFCs_bmimPF6_PR import configuration
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest
from pyomo.opt import TerminationCondition, SolverStatus

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
m1 = ConcreteModel()

m1.fs = FlowsheetBlock(default={"dynamic": False})

m1.fs.properties = GenericParameterBlock(default=configuration)

m1.fs.F101 = Flash(default={"property_package": m1.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

In [3]:
data = pd.read_csv('R32_bmimPF6_09_2.csv')
data2 = pd.read_csv('R125_bmimPF6_09_2.csv')

In [4]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [5]:
# Make outlet arrays
IL_x = np.zeros(len(data["1"]))
IL_y = np.zeros(len(data["1"]))
R32_x = np.zeros(len(data["1"]))
R32_y = np.zeros(len(data["1"]))
R125_x = np.zeros(len(data["1"]))
R125_y = np.zeros(len(data["1"]))

alpha1 = np.zeros(len(data["1"]))

m1.fs.F101.inlet.flow_mol.fix(10)
m1.fs.F101.inlet.temperature.fix(298)
m1.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.5) 
m1.fs.F101.inlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.pressure.fix(100000)
m1.fs.F101.vap_outlet.temperature.fix(298.15)

for i in range(len(data["0"])):
    
    m1.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m1.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m1.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(data["1"].iloc[i])
    m1.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(data["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(data2["0"].iloc[i])
    m1.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(data2["1"].iloc[i])

    m1.fs.F101.inlet.flow_mol.fix(10)
    m1.fs.F101.inlet.temperature.fix(298)
    m1.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.7) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.2) 
    m1.fs.F101.inlet.mole_frac_comp[0, "R125"].fix(0.1)

    m1.fs.F101.inlet.pressure.fix(100000)

    m1.fs.F101.vap_outlet.pressure.fix(100000)
    m1.fs.F101.vap_outlet.temperature.fix(298.15)

    try:
        m1.fs.F101.initialize(outlvl=idaeslog.CRITICAL)              

        # solve the model
        status = solver.solve(m1, tee = False)

        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):

            IL_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "bmimPF6"])
            IL_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "bmimPF6"])
            R32_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R32"]) 
            R32_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R32"])
            R125_x[i] = value(m1.fs.F101.liq_outlet.mole_frac_comp[0, "R125"])
            R125_y[i] = value(m1.fs.F101.vap_outlet.mole_frac_comp[0, "R125"])

            print(i)
            
            alpha1[i] = (R125_y[i]/R125_x[i])/(R32_y[i]/R32_x[i])

    except ValueError:
        alpha1[i] = 'NaN'
       
        
results = pd.DataFrame(alpha1)
results.to_csv('alpha_bmimPF6_09_2.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [6]:
print(alpha1)

[3.02782689 2.74385035 3.02783271 0.14865003 2.94703727 2.75591071
 2.99444874 2.95424852 3.09132373 3.06150786 3.035324   2.79927324
 3.01001623 2.82593649 2.98738182 2.14205305 2.96674426 3.067226
 3.05242911 3.01498174 2.68344994 2.87408373 3.07725736 3.00739779
 2.95866606 3.06717495 2.44418195 2.68498501 1.93393227 3.06044137
 2.94515996 3.08541033        nan 3.02471009 2.97971608 2.98067804
 2.97701607 2.31557588 2.95991572 2.62504873 2.98057037 3.13935835
 3.0650546  2.82398425 2.99064847 2.63854544 3.0448232  1.90318709
 2.98870077 3.11598047 3.07023268 3.10365882 2.87757644 2.94409171
 2.92229222 3.04336945 3.03572651 0.12148373 2.95734755 3.05133785
 3.02741441 3.02987482 3.03895295 2.73836417 2.94438433 2.96244496
 2.99660735 2.46191284 2.97217291 3.02192305 3.01466542 2.87767779
 3.00795398 2.74838406 2.45784038 3.01707307 2.86010643 2.76879745
 2.97605263 3.00196549 1.72558634 3.0518681  2.92271847 2.86339923
 2.97103556 2.81876628 2.98434897 2.83130692 2.9273193  2.991439